In [5]:
import numpy as np
import scipy
import utils
from time import time
from utils import QR_Factorization, EVD, SVD, Bidiagonal_fastMult

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Problem 1: SVD by Two-Phase Approach

## Phase-I: Golub-Kahan Bidiagonalization

In [6]:
A = np.array([[0, 0, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 0],
              [2, 5, 0, 0],
              [0, 0, 0, 0],
              [0, 0, 0, 0]], dtype=np.float64)
B, Qt, P = SVD.svd_phaseI(A)
print(B)


[[ 2. -5.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


## Phase-II

In [7]:
m = n = 1024
A = np.random.rand(m,n)

**Test Fast Multiplication for A@B, where B is upper bidiagonal matrix.**

Numpy @ might use multi-thread to accelerate the computation. But our implementation is O(n^2) which is theoretically more effcient.

In [8]:
B, _, _ = SVD.svd_phaseI(A)

Test for `fastMult_upper_bidiagonal`

In [9]:
numpy_mul_begin = time()
for i in range(1000):
    A@B
numpy_mul_end = time()
print("{:.4f}s".format(numpy_mul_end - numpy_mul_begin))

19.3885s


In [10]:
fastMul_begin = time()
for i in range(1000):
    Bidiagonal_fastMult.fastMult_upper_bidiagonal(A, B)
fastMul_end = time()
print("{:.4f}s".format(fastMul_end - fastMul_begin))

25.3285s


Test for `upper_fastMult_lower_bidiagonal`


In [11]:
numpy_mul_begin = time()
for i in range(1000):
    B@B.T
numpy_mul_end = time()
print("{:.4f}s".format(numpy_mul_end - numpy_mul_begin))

13.8027s


In [12]:
fastMul_begin = time()
for i in range(1000):
    Bidiagonal_fastMult.upper_fastMult_lower_bidiagonal(B, B.T)
fastMul_end = time()
print("{:.4f}s".format(fastMul_end - fastMul_begin))

7.9790s


Test for `qr_tridiagonal_by_Givens` and `qr_lower_bidiagonal_by_Givens`

In [13]:
begin = time()
for i in range(100):
    QR_Factorization.qr_tridiagonal_by_Givens(B.T, return_Givens=True)
end = time()
print("{:.4f}s".format(end-begin))

4.1187s


In [14]:
begin = time()
for i in range(100):
    QR_Factorization.qr_lower_bidiagonal_by_Givens(B.T, return_Givens=True)
end = time()
print("{:.4f}s".format(end-begin))

1.0825s


**Test SVD**

Choose the parameter phaseII as 'A', 'B1', 'B2' to test different implementations of phase II

In [27]:
m = 150
n = 150
A = np.random.rand(m,n)
A[n-50:n] = A[n-50:n] * 1000
from utils.SVD import accuracy_test

In [28]:
U, S, Vt = SVD.svd(A, phaseII='A')
# U, S, Vt = SVD.svd(A, phaseII='A2')
# U, S, Vt = SVD.svd(A, phaseII='B')
# U, S, Vt = SVD.svd(A, phaseII='B2')
# U, S, Vt = SVD.svd(A, phaseII='C')
_, Ss, _  = scipy.linalg.svd(A, full_matrices=False)

accuracy_test(U, S, Vt, Ss, acc=1e-8)

phaseI: 0.0301s
phaseII: 0.4207s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
6.535660901363372e-11


**Accuracy Test:**

Modify acc as you like!

In [29]:
accuracy_test(U, S, Vt, Ss, acc=1e-8)

Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
6.535660901363372e-11


**Scipy SVD**

In [30]:
U, Ss, Vt  = scipy.linalg.svd(A, full_matrices=False)
# print(np.abs(U@np.diag(S)@Vt - A))
acc = 1e-8
print("Percentage of entrices successfully recovered by SVD with accuracy: {}".format(acc))
print(np.sum(np.abs(U@np.diag(Ss)@Vt - A)< acc) / (n*m) * 100, "%")
# U, S, Vt

Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %


## Test for p2

In [34]:
m = n = 100
k = 4
A = np.array(np.diag([1/2]*n, 0)+np.diag([1/4]*(n-1), 1)+np.diag([1/4]*(n-1), -1))

A = np.linalg.matrix_power(A, k)
U, S, Vt = SVD.svd(A, phaseII="B2")
_, Ss, _  = scipy.linalg.svd(A, full_matrices=False)

accuracy_test(U, S, Vt, Ss, acc=1e-8)

phaseI: 0.0105s
phaseII: 0.7890s
Percentage of entrices successfully recovered by SVD with accuracy: 1e-08
100.0 %
Percentage of singular values with accuracy: 1e-08
100.0 %
Max error of singular values:
8.584226131177716e-09


Modify acc as you like!

In [32]:
accuracy_test(U, S, Vt, Ss, acc=1e-15)

Percentage of entrices successfully recovered by SVD with accuracy: 1e-15
100.0 %
Percentage of singular values with accuracy: 1e-15
81.0 %
Max error of singular values:
1.2323475573339238e-14
